In [1]:
import polars as pl
import polars.selectors as cs

Part 1

In [ ]:
# from io import StringIO
# IN_ = """123 328  51 64 
#  45 64  387 23 
#   6 98  215 314
# *   +   *   +  """
# IN = StringIO(IN_)
IN = "d6_in.txt"
print(
  pl.read_csv(IN, has_header=False)
  .with_columns(
    pl.col("column_1")
    .str
    .strip_chars(" ")
    .str
    .replace_all(r"\s+", ",")
    .str
    .split(",")
    .list
    .to_struct(),
  )
  .unnest("column_1")
  .reverse()
  .transpose()
  .rename({"column_0": "ops"})
  .with_columns(cs.contains("column_").cast(pl.Int64))
  .with_columns(
    total=
    pl.when(pl.col("ops") == "+")
    .then(pl.reduce(function=lambda acc, x: acc + x, exprs=cs.contains("column_")))
    .otherwise(pl.reduce(function=lambda acc, x: acc * x, exprs=cs.contains("column_")))
  )
  .select(pl.col("total").sum())
)

shape: (1, 1)
┌───────────────┐
│ total         │
│ ---           │
│ i64           │
╞═══════════════╡
│ 5877594983578 │
└───────────────┘


Part 2

In [47]:
# 123 328  51 64 
#  45 64  387 23 
#   6 98  215 314
# *   +   *   +
# from io import StringIO
# IN_ = """123 328  51 64 
#  45 64  387 23 
#   6 98  215 314
# *   +   *   +  """
# IN = StringIO(IN_)
IN = "d6_in.txt"
print(
    pl.read_csv(IN, has_header=False)
    .with_columns(
        pl.col("column_1")
        # do a first pass, repeat op except one space
        .str
        .replace_all(r"(?<o>\*|\+)\s\s(?<p>\+|\*)", "$o$o $p")
        .str
        .replace_all(r"(?<o>\*|\+)\s\s\s(?<p>\+|\*)", "$o$o$o $p")
        .str
        .replace_all(r"(?<o>\*|\+)\s\s\s\s(?<p>\+|\*)", "$o$o$o$o $p")
        # do a second pass (above we did o...p, now we need p...o)
        .str
        .replace_all(r"(?<p>\*|\+)\s\s(?<o>\+|\*)", "$p$p $o")
        .str
        .replace_all(r"(?<p>\*|\+)\s\s\s(?<o>\+|\*)", "$p$p$p $o")
        .str
        .replace_all(r"(?<p>\*|\+)\s\s\s\s(?<o>\+|\*)", "$p$p$p$p $o")
        # handle the end
        .str
        .replace_all(r"(?<o>\*|\+)\s\s$", "$o$o$o")
        .str
        .replace_all(r"(?<o>\*|\+)\s\s\s$", "$o$o$o$o")
        .str
        .replace_all(r"(?<o>\*|\+)\s\s\s\s$", "$o$o$o$o$o")
        # add dummy first and last columns (lines after transpose)
        # needed for the last shoft res_tmp to res
        .str
        .replace_all(r"^(\d|\s)", "1 $1")
        .str
        .replace_all(r"^(\+|\*)", "* $1")
        .str
        .replace_all(r"(\d|\s)$", "$1 1")
        .str
        .replace_all(r"(\+|\*)$", "$1 *")
        # fill with zeros
        .str
        .replace_all(" ", "0")
        # ...
        .str
        .split("")
        .list
        .to_struct()
    )
    .reverse()
    .unnest("column_1")
    .transpose()
    .rename({"column_0": "ops"})
    .with_columns(
        pl.concat_str(cs.contains("column_"))
        .str
        .reverse()
        .str
        .strip_chars("0")
        .alias("nums")
    )
    .with_columns(
        pl.when(pl.col("nums") == "")
        .then(None)
        .otherwise(pl.col("nums"))
        .alias("nums")
        .cast(pl.Int64)
    )
    .drop(cs.contains("column_"))
    .with_columns(
        res_tmp=
        pl.when(
            (pl.col("ops") == "*")
            & (pl.col("ops").shift(-1) == "*")
            & (pl.col("ops").shift(-2) == "*")
            & (pl.col("ops").shift(-3) == "*")
            & (pl.col("ops").shift(-4) == "0")
        )
        .then(
            pl.col("nums")
            * pl.col("nums").shift(-1)
            * pl.col("nums").shift(-2)
            * pl.col("nums").shift(-3)
        )
        .when(
            (pl.col("ops") == "+")
            & (pl.col("ops").shift(-1) == "+")
            & (pl.col("ops").shift(-2) == "+")
            & (pl.col("ops").shift(-3) == "+")
            & (pl.col("ops").shift(-4) == "0")
        )
        .then(
            pl.col("nums")
            + pl.col("nums").shift(-1)
            + pl.col("nums").shift(-2)
            + pl.col("nums").shift(-3)
        )
        .when(
            (pl.col("ops") == "*")
            & (pl.col("ops").shift(-1) == "*")
            & (pl.col("ops").shift(-2) == "*")
            & (pl.col("ops").shift(-3) == "0")
        )
        .then(
            pl.col("nums")
            * pl.col("nums").shift(-1)
            * pl.col("nums").shift(-2)
        )
        .when(
            (pl.col("ops") == "+")
            & (pl.col("ops").shift(-1) == "+")
            & (pl.col("ops").shift(-2) == "+")
            & (pl.col("ops").shift(-3) == "0")
        )
        .then(
            pl.col("nums")
            + pl.col("nums").shift(-1)
            + pl.col("nums").shift(-2)
        )
        .when(
            (pl.col("ops") == "*")
            & (pl.col("ops").shift(-1) == "*")
            & (pl.col("ops").shift(-2) == "0")
        )
        .then(
            pl.col("nums")
            * pl.col("nums").shift(-1)
        )
        .when(
            (pl.col("ops") == "+")
            & (pl.col("ops").shift(-1) == "+")
            & (pl.col("ops").shift(-2) == "0")
        )
        .then(
            pl.col("nums")
            + pl.col("nums").shift(-1)
        )
        .when(
            (pl.col("ops") == "*")
            & (pl.col("ops").shift(-1) == "0")
        )        
        .then(
            pl.col("nums")
        )
        .when(
            (pl.col("ops") == "+")
            & (pl.col("ops").shift(-1) == "0")
        )
        .then(
            pl.col("nums")
        )
    )
    .with_columns(
        res=
        pl.when(pl.col("ops") == "0")
        .then(pl.col("res_tmp").shift(-1))
    )
    .select(pl.col("res").sum())
)

shape: (1, 1)
┌────────────────┐
│ res            │
│ ---            │
│ i64            │
╞════════════════╡
│ 11159825706149 │
└────────────────┘
